In [ ]:
import os
import sys
import time
import numpy as np
import pandas as pd

import anndata as sc
import snapatac2

sys.path.insert(0, '/u/home/f/f7xiesnm/project-zipursky/code/scroutines')
from scroutines import basicu
import atac_utils

In [ ]:
snapatac2.__path__

In [ ]:
snapatac2.__version__

In [ ]:
%%time
f_rna = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/L23_allmultiome_proc_P6toP21_NRDR.h5ad'
adata = sc.read(f_rna)
adata

In [ ]:
organized_sample_dict = atac_utils.get_organized_sample_dict()

In [ ]:
out_dir = '/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/atac_fragments/covr_snap_v2/'

In [ ]:
%%time

SHUFFLE = False
n_type = 5


ti = time.time()

for cond_code, samples in organized_sample_dict.items(): 
    cond_name = atac_utils.decode_cond(cond_code)
    print(cond_code, cond_name, samples)
    
    # get sub
    adatasub = adata[adata.obs['cond']==cond_name]
    cells_rna = adatasub.obs.index.values
    
    # get RNA type
    x0 = adatasub.obsm['pca_p17on'][...,0]
    # x_shuff
    if SHUFFLE:
        x = x0[np.random.choice(len(x0), size=len(x0), replace=False)]
    else:
        x = x0
    cells_rna_type = pd.qcut(x, n_type, labels=False)
    
    # prep ATAC
    for sample in samples:
        print(sample, time.time()-ti)
        # atac all cells
        f = f'/u/home/f/f7xiesnm/project-zipursky/v1-bb/v1/data/v1_multiome/atac_fragments/frag_snap/ATAC_{sample}.h5ad'
        adata_atac = sc.read_h5ad(f)

        cell_names_raw = adata_atac.obs.index.values
        cell_names_atac = np.char.add(cell_names_raw.astype(str), f'-{sample}-2023 Multiome-{cond_code}-0')
        adata_atac.obs.index = cell_names_atac
        
        # check compatibility
        cell_names_both = np.intersect1d(cell_names_atac, cells_rna)
        print(len(cell_names_both)/len(cells_rna)*100)
        
        # assign RNA type
        adata_atac = adata_atac[cell_names_both]
        adata_atac.obs['rna_type'] = np.char.add('RT', cells_rna_type[basicu.get_index_from_array(cells_rna, cell_names_both)].astype(str)) 

        # export 
        snapatac2.ex.export_coverage(adata_atac, groupby='rna_type', 
                                     normalization=None, 
                                     counting_strategy='insertion', # not implemented yet'paired-insertion',
                                     out_dir=out_dir, prefix=f'covr_{sample}_') 
        # adata_atac.close()

    #     break
    # break